In [1]:
# Import libraries.
import pandas as pd
import numpy as np
from pymatgen.core.composition import Composition
from KmdPlus import StatsDescriptor, formula_to_composition 
import pickle
import itertools
import copy
import os
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import math
from scipy.spatial import distance_matrix
import shutil

# Read all templates.
MP_stable = pd.read_pickle("data_set/MP_stable_20211107.pd.xz")

# Element-level descriptors of shape (94, 58).
element_features = pd.read_csv("data_set/element_features.csv", index_col= 0)

# Load test data (90 crystals).
test_data = pd.read_pickle("data_set/all_searching_targets_20211107_with_predictions.pd.xz")

# Load the pre-trained models.
with open("data_set/CSPML_models.xz", "rb") as f:
    models = pickle.load(f)
    
# Load stats.
cmpfgp_stable_meanstd = np.load("data_set/cmpfgp_stable_meanstd_20211107.npy") 

# Load element dissimilarity.
element_dissimilarity = np.load('data_set/element_dissimilarity.npy')

Metal device set to: Apple M1 Max


2024-07-03 12:47:33.698045: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-03 12:47:33.698188: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
# ensemble models.
def ensemble_models(X, models):
    y_pred = np.array([models[i].predict(X, verbose=0)[:,1] for i in range(len(models))])
    
    return y_pred.mean(0)

# Formula to ratio label.
def formula_to_ratiolabel(formula):
    # Convert chemical formulas to compositions.
    weight = np.array([formula_to_composition(formula[i]) for i in range(len(formula))])
    
    ratio_label = []
    for i in range(len(formula)):
        sorted_weight = np.sort(weight[i])[::-1]
        comp = Composition(formula[i])
        comp_ratio = comp.num_atoms * sorted_weight
        x = [int(round(comp_ratio[j])) for j in range(len(comp))]
        gcd_x = math.gcd(*x) 
        # For collection in the case like "O2", "Na2O2".
        if gcd_x != 1:
            x = [int(round(x[k]/gcd_x)) for k in range(len(x))]
        else:
            pass
        # Get ratio label for collected x.
        label = ""
        for j in range(len(x)):
            label += f"{x[j]}:"
        # Save results.
        ratio_label.append(label[:-1])
    
    return np.array(ratio_label, dtype = "object")

# Screening for CSPML.
def Screening_candidates(query_formula, top_K, templates, cutoff = 0.5, element_features = element_features,
                        meanstd = cmpfgp_stable_meanstd, models = models):
    
    # Calculate cmpfgp for quary formula.
    query_cmpfgp = StatsDescriptor(query_formula, element_features)
    query_cmpfgp = (query_cmpfgp - meanstd[0])/meanstd[1] # scaling.
    
    # Calculate ratio label.
    query_ratiolabel = formula_to_ratiolabel(query_formula)
    
    # Make predictions.
    predictions = []

    for i in range(len(query_formula)):
        ix = np.where(templates.comp_ratio_label.values == query_ratiolabel[i])[0]

        if len(ix) < 1:
            print(f"None of the candidates had the same composition ratio as {query_formula[i]}.")
            topK_id, topK_pred, topK_formula = [], 0, []
        else:
            x = templates.iloc[ix]
            # Composition fingerprint for x.
            x_cmpfgp = x.cmpfgp.values
            x_cmpfgp = np.array([x_cmpfgp[i] for i in range(len(x_cmpfgp))])
            X = np.abs(x_cmpfgp - query_cmpfgp[i,:])
            y_pred = ensemble_models(X, models)
            topK_ix = np.argsort(y_pred)[::-1][:top_K]

            topK_id = x.materials_id.values[topK_ix]
            topK_pred = y_pred[topK_ix]
            topK_formula = x.pretty_formula.values[topK_ix]

            survived = (topK_pred > cutoff)

            if sum(survived) < 1:
                    print(f"None of the candidates had the class probabilities greater than {cutoff} at {query_formula[i]}.")
                    topK_id, topK_pred, topK_formula = [], 0, []
            else:
                topK_id, topK_pred, topK_formula = topK_id[survived], topK_pred[survived], topK_formula[survived]


        prediction_result = {"query_formula":query_formula[i],"topK_formula":topK_formula,
                                 "topK_id":topK_id,"topK_pred":topK_pred}
        predictions.append(prediction_result)
        
    return predictions

# CSPML.
def Structure_prediction(query_formula, top_K, templates, cutoff = 0.5, element_features = element_features,
            meanstd = cmpfgp_stable_meanstd, models = models, element_dissimilarity = element_dissimilarity,
            SI = False, save_cif = False, save_cif_filename = ""):
    
    # Screening top_K candidates using pre-trained model for each query formula.
    screened = Screening_candidates(query_formula, top_K, templates, cutoff, element_features,
                        meanstd, models)
    
    element_symbol = element_features.index.values
    predictions = []

    for i in range(len(query_formula)):

        predicted_structures = []
        scr_num = len(screened[i]["topK_id"])

        if scr_num == 0:
            pass

        else:
            for j in range(scr_num):

                # The ith query formula.
                vec = formula_to_composition(query_formula[i])
                N_ele = sum(vec != 0)
                comp_index = np.argsort(vec)[::-1][:N_ele]

                # Top-jth suggested formula for ith query formula.
                sug_formula = screened[i]['topK_formula'][j]
                vec_sug = formula_to_composition(sug_formula)
                comp_sug_index = np.argsort(vec_sug)[::-1][:N_ele]

                # Composition of ith fomula (quary & suggested) and it's unique composition ratio.
                comp = np.sort(vec)[::-1][:N_ele]
                keys = np.sort(list(set(comp)))[::-1]

                # Grouping composition-index(=element species) according to unique composition ratio.
                group_index = []
                group_sug_index = []
                for k in range(0, len(keys)):
                    x = (comp == keys[k])
                    group_index.append(comp_index[x])
                    group_sug_index.append(comp_sug_index[x])

                # Find out elements-replacement that minimize element-dissimilarity and make dict showing replacement.
                replacement = []
                for l in range(0, len(keys)):
                    # Replacement is unique.
                    if len(group_index[l]) == 1:
                        replacement.append(group_sug_index[l])
                    # Replacement is not unique.
                    else :
                        seq = group_sug_index[l]
                        pmt = list(itertools.permutations(seq))
                        K = len(pmt)
                        dis_sum = np.zeros(K)
                        for m in range(0, K):
                            dis_sum[m] = sum(element_dissimilarity[group_index[l], pmt[m]]) # element_dissimilarity.
                        replacement.append(np.array(pmt[np.argmin(dis_sum)]))
                rep_index = np.concatenate(replacement)
                q_ele = element_symbol[comp_index]
                rep_ele = element_symbol[rep_index]
                rep_dict = dict(zip(rep_ele,q_ele))

                # Generating top-jth candidate structure for ith query formula.
                query_str = copy.deepcopy(templates[templates.materials_id.values == screened[i]["topK_id"][j]].structure[0])
                query_str.replace_species(rep_dict)
                predicted_structures.append(query_str)

                # Save the structure object as a .cif file into dir = filename (if save_cif=True).
                if save_cif:
                    text =  f"{save_cif_filename}/{query_formula[i]}_{j+1}.cif"
                    query_str.to(filename=text)
                else:
                    pass

        predictions.append(predicted_structures)

    # Return the predicted structures (+ optionally the supplementary information of the predicted structures).
    if SI:
        return predictions, screened
    else:
        return predictions

In [3]:
# Create a directory for saving results (results should be same as cif_files_for_90crystals/predicted_structures (pre DFT)).
new_dir = "CSPML_test90"
if os.path.exists(new_dir):
    shutil.rmtree(new_dir)
os.mkdir(new_dir)

# Make CSPML prediction.
predictions, screened = Structure_prediction(test_data.pretty_formula.values, 10, MP_stable,
                    SI = True, save_cif=True, save_cif_filename=new_dir)

2024-07-03 12:48:14.818388: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-07-03 12:48:14.877639: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-07-03 12:48:15.034916: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-07-03 12:48:15.160049: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-07-03 12:48:15.291992: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-07-03 12:48:15.432128: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


None of the candidates had the same composition ratio as NaCaAlPHO5F2.
None of the candidates had the class probabilities greater than 0.5 at MgB7.
None of the candidates had the class probabilities greater than 0.5 at Ba2CaSi4(BO7)2.
None of the candidates had the same composition ratio as K5Ag2(AsSe3)3.
None of the candidates had the same composition ratio as Na(WO3)9.
None of the candidates had the same composition ratio as Li6V3P8O29.
None of the candidates had the same composition ratio as Mg3Si2H4O9.
None of the candidates had the class probabilities greater than 0.5 at Y4Si5Ir9.


In [4]:
# Load test (ground truth) structure fingerprints (see Create_strcmp_fgp.ipynb for details).
test_strfgp = np.load('data_set/strfgp_test_20211107.npy')

# Get structure fingerprints for the predicted structures (pre DFT).
predicted_strfgp = []

for i in range(len(screened)):
    strfgp = []
    len_j = len(predictions[i])
    if len_j==0:
        pass
    else:
        for j in range(len_j):
            strfgp.append(MP_stable[MP_stable.index.values == screened[i]["topK_id"][j]].strfgp[0] )
            
    predicted_strfgp.append(strfgp)
    
# Get dissimilarity for predicted (pre DFT) vs true.
predicted_dissim = []

for i in range(len(predicted_strfgp)):
    dissim = []
    len_j = len(predicted_strfgp[i])
    if len_j == 0:
        dissim.append(1000)
    else:
        for j in range(len_j):
            dissim.append(np.sum((test_strfgp[i, :] - predicted_strfgp[i][j])**2)**(1/2) )
    
    predicted_dissim.append(dissim)
    
# Get min dissim for each i.
predicted_dissim_min = np.array([np.min(np.array(predicted_dissim[i])) for i in range(len(predicted_dissim))])
predicted_formula = np.array([screened[i]["query_formula"] for i in range(len(screened))])

# Sumarize and save the results.
tau = 0.2

template_dissim = pd.DataFrame(np.array([predicted_dissim_min, (predicted_dissim_min <= tau)]).T,
            columns=["Min. dissim (top10)", "<=0.2"], index=predicted_formula)

template_dissim.to_csv("template_dissim.csv")

print(f"dissim<=0.2: {int(sum(template_dissim.iloc[:,1]))}/{template_dissim.shape[0]}")

template_dissim

dissim<=0.2: 51/90


,Min. dissim (top10),<=0.2
C,2.534663e+00,0.0
Si,3.500210e-17,1.0
GaAs,3.877854e-16,1.0
ZnO,4.216784e-01,0.0
BN,1.726447e+00,0.0
...,...,...
VPt3,4.285599e-02,1.0
SmVO4,5.189530e-02,1.0
VCl5,1.235371e-01,1.0
LaSi2Ni9,2.569991e-02,1.0


In [5]:
# Load relaxed predictions (after DFT, structural optimizations by DFT were performed on a separate machine).
relaxed = pd.read_pickle('data_set/CSML_90_opt_results.pd.xz')
relaxed_converged = relaxed[relaxed.converged == True]
relaxed_formula = np.unique(relaxed_converged.target_formula.values)

# Save predictred structures (after DFT, results should be same as cif_files_for_90crystals/predicted_structures (after DFT)).
new_dir = "predicted_structures (after DFT)"
if os.path.exists(new_dir):
    shutil.rmtree(new_dir)
os.mkdir(new_dir)
        
for i in range(len(relaxed_formula)):
    str_formula = relaxed_converged[relaxed_converged.target_formula == relaxed_formula[i]]
    
    for j in range(str_formula.shape[0]):
        str_x = str_formula.final_structure[j]
        text =  f"{new_dir}/{relaxed_formula[i]}_{str_formula.index[j]}.cif"
        str_x.to(filename=text)
        
# Load libraries for calculating strfgp.
from matminer.featurizers.site import CrystalNNFingerprint  # matminer version = 0.6.2 (later version will give same calculation results).
from matminer.featurizers.structure import SiteStatsFingerprint
# Parallel calculation.
import joblib

# Site featurizer.
cnnf = CrystalNNFingerprint.from_preset('ops', distance_cutoffs=None, x_diff_weight=0)

def parallel_cnnf(featurizer, str_x):
    return np.array(joblib.Parallel(n_jobs=-1)(joblib.delayed(featurizer)(str_x, i) for i in range(len(str_x.sites))))

# SiteStats.
def SiteStats(site_fgps):
    return np.array([site_fgps.mean(0), site_fgps.std(0), site_fgps.min(0), site_fgps.max(0)]).T.flatten()
        
# Calculate structure finger prints for the relaxed structures.
dft_fgp = []
for i in range(len(relaxed_formula)):
    x = relaxed_converged[relaxed_converged.target_formula == relaxed_formula[i]]
    str_x = np.array([SiteStats(parallel_cnnf(cnnf.featurize, x.final_structure[j])) for j in range(x.shape[0])])
    dft_fgp.append(str_x)
    
true_fgp = []
for i in range(len(relaxed_formula)):
    x = test_data[test_data.pretty_formula == relaxed_formula[i]].structure[0]
    true_fgp.append(SiteStats(parallel_cnnf(cnnf.featurize, x)))

# Get dissimilarity for predicted (pre DFT) vs true.
dissim = np.array([np.min(np.sum((true_fgp[i] - dft_fgp[i])**2,1)**(1/2)) for i in range(len(relaxed_formula))])

# Sumarize and save the results.
template_dissim_dft = pd.DataFrame(np.array([dissim, (dissim <= tau)]).T,
            columns=["Min. dissim (top10)", "<=0.2"], index=relaxed_formula)

template_dissim_dft.to_csv("template_dissim_dft.csv")

print(f"dissim<=0.2: {int(sum(template_dissim_dft.iloc[:,1]))}/{template_dissim.shape[0]}")

template_dissim_dft

dissim<=0.2: 59/90


,Min. dissim (top10),<=0.2
Ag8GeS6,0.118303,1.0
Al2CoO4,0.384012,0.0
Al2O3,0.001500,1.0
AlH12(ClO2)3,0.057771,1.0
BN,0.042920,1.0
...,...,...
ZnO,0.423429,0.0
ZnSb,1.443540,0.0
Zr4O,2.096172,0.0
ZrO2,0.118792,1.0


In [6]:
# Save test (ground truth) structures as cif files (results should be same as cif_files_for_90crystals/ground_truth_structures).
new_dir = "ground_truth_structures"
if os.path.exists(new_dir):
    shutil.rmtree(new_dir)
os.mkdir(new_dir)

for i in range(test_data.shape[0]):
    str_x = test_data.structure[i]
    text =  f"{new_dir}/{test_data.pretty_formula[i]}.cif"
    str_x.to(filename=text)